In [29]:
import numpy as np
import cvxopt
from cvxopt import matrix, solvers
import pandas as pd


rho = 1.
a = 0.25
b = 1





def kernel(x,y):
    x_d = x.data
    y_d = y.data
#     x_a = csr_matrix.power(x,a)
#     y_a = csr_matrix.power(y,a)
#     s = x_a-y_a
#     s = csr_matrix.power(s),b)
#     s = np.sum(s)
    return np.exp(-rho*s)

def kernel_matrix(X):
    n = X.shape[0]
    K = np.zeros((n,n))
    for i in range(n):
        if (i % 500 == 0):
            print(i)
        for j in range(n):
            K[i,j] = kernel(X[i,:],X[j,:])
    return K

def transform_dual(K, y, C=300,):
    # Here, we keep lambda as the variable. We need to make A bigger to account for the 2 inequalities on lambda
    n = X.shape[0]
    Q = matrix(np.diag(y)) * K * matrix(np.diag(y))
    p = - np.ones(n)
    G = np.zeros((2 * n, n))
    h = np.zeros(2 * n)
    G[:n, :n] = np.eye(n)
    G[n:2 * n, :n] = -np.eye(n)
    h[:n] = C * np.ones(n)
    return matrix(Q), matrix(p), matrix(G), matrix(h)

# def transform_dual2(X,y,C=.5):
#     # Here, we keep lambda as the variable. We need to make A bigger to account for the 2 inequalities on lambda
#     n = X.shape[0]
#     X_m = matrix(X.values)
#     P = X_m * X_m.T
#     q = - y * np.ones(n)
#     G = np.zeros((2 * n, n))
#     h = np.zeros(2 * n)
#     G[:n, :] = matrix(np.diag(y))
#     G[n:2 * n, :] = matrix(-np.diag(y))
#     h[:n] = C * np.ones(n)
#     return matrix(P), matrix(q), matrix(G), matrix(h)


def SVM(K, y, C=300):
    Q, p, G, h = transform_dual(K, y, C)
    w_sol = solvers.qp(Q, p, G, h)
    print(w_sol)
    return np.dot(w_sol['x'].T,np.diag(y))


def predict(w, x ,K):
    ker = np.array([kernel(x,X[i]) for i in range(X.shape[0])])
    return np.sign(w * ker)

def pred_onevsall(models,x,X,y):
    cls,margin = 0,0 
    ker = np.array([kernel(x,X[i]) for i in range(X.shape[0])])
    for i,m in models.items():
        a = np.dot(m, ker)
        if np.abs(a) > margin and a > 0:
            cls,margin = i, np.abs(a)
    return cls

def onevsallSVM(X, y, C=300):
    models = {}
    for cls in np.unique(y):
        X_train = X.copy()
        y_train = pd.Series([-1 if yval == cls else 1 for yval in y])
        models[cls] = SVM(X_train, y_train, C=.5)
    return models


def onevsoneSVM(X, y, C=300):
    for class1 in np.unique(y):
        for class2 in np.unique(y):
            if class1 != class2:
                X_train = pd.concat((X.loc[y == class1], X.loc[y == class2]))
                y_train = pd.concat((y.loc[y == class1]), y.loc(y == class2))

def error_rate(ypred,ytrue):
    return np.mean(1*(ypred != ytrue))

In [31]:
#data = pd.read_csv('../data/Xtr.csv', sep=',',header = None)
#data = data.iloc[:,:3072]
data_train = pd.read_pickle('./data/histo_16s_train')
data_test = pd.read_pickle('./data/histo_16s_test')

label = pd.read_csv('./data/Ytr.csv',sep = ',')

In [32]:
print(data_train.shape,data_test.shape)

(5000, 4096) (2000, 4096)


In [4]:
data_train.index = range(5000)

In [6]:
tr = data_train.to_dense()

In [41]:
ind = np.random.binomial(1,0.8,5000)
print(np.sum(ind))

3948


In [9]:
from scipy.sparse import csr_matrix
tr = csr_matrix(tr.values)

In [43]:
train = data_train.loc[ind == 1]
train_label = label.loc[ind == 1].iloc[:,1].values

test = data_train.loc[ind == 0]
test_label = label.loc[ind == 0].iloc[:,1].values

In [30]:
kernel_matrix(tr)

0


KeyboardInterrupt: 

In [ ]:
models = onevsallSVM(train,train_label, C = 100)

In [223]:
print(models)

{0: array([[  9.30453702e-07,   3.13003634e-07,   1.30576026e-07, ...,
          4.96378299e-01,   9.89646119e-07,   1.99636069e-07]]), 1: array([[  9.93405123e-08,   4.99999580e-01,   4.99999375e-01, ...,
          4.99999988e-01,   1.11893000e-07,   5.85319948e-08]]), 2: array([[ 0.01717411,  0.00276345,  0.00538051, ...,  0.00299182,
         0.13237068,  0.03097401]]), 3: array([[ 0.44653776,  0.01049439,  0.5       , ...,  0.01836749,
         0.01067936,  0.00831636]]), 4: array([[  5.03505634e-02,   8.96422166e-03,   7.69588267e-05, ...,
          8.19417292e-05,   7.00915565e-03,   2.30495694e-02]]), 5: array([[ 0.02051281,  0.01439929,  0.09606096, ...,  0.00732323,
         0.00783258,  0.5       ]]), 6: array([[  4.69279492e-03,   1.17911005e-02,   2.58445609e-03, ...,
          8.34438237e-08,   2.57623293e-02,   8.60081476e-03]]), 7: array([[ 0.02323794,  0.10373199,  0.01230912, ...,  0.00177595,
         0.49999999,  0.01227343]]), 8: array([[ 0.49999999,  0.00129469,  0

In [226]:
ypred = []
for i in range(5000 - np.sum(ind)):
    ypred.append(pred_onevsall(models, test.iloc[i,:], train,train_label))

ypred = np.array(ypred)

In [227]:
error_rate(ypred,test_label)

0.9150197628458498

In [154]:
len(ind)

5000

In [64]:
a*b

TypeError: incompatible dimensions

In [66]:
print(a * b.T)

[ 1  0  1]
[ 2  0  2]
[ 3  0  3]



In [61]:
np.dot(a,b.T)

4

In [67]:
d = {}
d[1] = 1
d[2] = 3

In [68]:
d

{1: 1, 2: 3}

In [73]:
for i,n in d.items():
    print(i,n)

1 1
2 3


In [149]:
yp = np.array([0,1,2,3])
yt = np.array([0,1,3,4])
1*(yp-yt == 0)

array([1, 1, 0, 0])